In [5]:
import sys
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install python-slugify


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [22]:
import pandas as pd
import os
from slugify import slugify

# Strukturvorlage laden, um allowlist für Attribute zu erhalten
# !! diese muss vorher exportiert worden sein
attribute_definitions_dir = '../../attribute-definitions/import/out/species-attribute-definitions.csv'
attribute_definitions_df = pd.read_csv(attribute_definitions_dir)

# Verzeichnis, in dem sich alle Excel-Dateien befinden
directory = '../'

# Definiere die Zielspeicherort für die CSV-Dateien
output_directory = './out/attributes'

# Lege sicher, dass der Ordner existiert, ansonsten erstelle ihn
os.makedirs(output_directory, exist_ok=True)

# Alle Dateien im Verzeichnis durchgehen
for filename in os.listdir(directory):
    # Überprüfen, ob die Datei eine .xlsx-Datei ist
    if filename.endswith('.xlsx'):
        file_path = os.path.join(directory, filename)
        
        # Excel-Datei laden und das Sheet "Portrait" einlesen
        try:
            df = pd.read_excel(file_path, sheet_name='Portrait')
        except Exception as e:
            print(f"Fehler beim Laden der Datei {filename}: {e}")
            continue

        # Überprüfen, ob die notwendigen Spalten vorhanden sind
        #required_columns = ['scientific_name', 'Zweck', 'Anmerkungen', 'Quelle']
        #if not all(col in df.columns for col in required_columns):
        #    print(f"Erforderliche Spalten fehlen in {filename}")
        #    continue

        # Artenportrait aus dem Arbeitsblatt "Artenportrait" laden
        try:
            species_name = df.loc[0, 'Text']  # Annahme: Der lateinische Name steht in der ersten Zeile unter 'Text'
        except Exception as e:
            print(f"Fehler beim Laden des Arbeitsblatts 'Artenportrait' in {filename}: {e}")
            continue
        
        #print(species_name)
        # Headernamen anpassen
        df.columns = [
            'sort_1', 'sort_2', 'category_1', 'category_2', 'field_name',
            'description', 'explanation', 'attribute_value', 'sources'
        ]
        
        # slug erstellen zur Zuordnung zu den AttributeDefinitions
        df['tmp'] = df['category_1'].fillna('') + ' ' + df['category_2'].fillna('') + ' ' + df['field_name'].fillna('')
        df['attribute_slug'] = df['tmp'].apply(lambda x: slugify(x, separator="_", replacements=[('ä', 'ae'), ('ö', 'oe'), ('ü', 'ue'), ('ß', 'ss')]))
       
        # irrelevante Columns droppen
        df = df.drop(columns=['tmp', 'sort_1', 'sort_2', 'category_1', 'category_2', 'field_name', 'description', 'explanation'])

        # filter by allowed attributes from schema file
        df = df[df['attribute_slug'].isin(attribute_definitions_df['slug'])]

        df['species'] = species_name

        # Hinzufügen einer leeren 'id'-Spalte
        df['id'] = None

        # reorder columns to fit database col order for import
        df = df[['id', 'attribute_value', 'sources', 'species', 'attribute_slug']]

        # print(df.head())
        
        # Zielpfad für die CSV
        #output_csv_path = os.path.join(output_directory, f"{filename.split('.')[0]}_species_plant_relationship.csv")
        output_csv_path = os.path.join(output_directory, f"{species_name}_attributes.csv")
        
        # CSV-Datei exportieren
        df.to_csv(output_csv_path, index=False)

        print(f'Die Attribute für {filename} wurde erfolgreich exportiert nach: {output_csv_path}')


Die Attribute für 20241105_Bärtige Sandbiene_Andrena barbilabris_AM.xlsx wurde erfolgreich exportiert nach: ./out/attributes/Andrena barbilabris_attributes.csv
Die Attribute für 20241105_Haussperling_Passer_domesticus_AM.xlsx wurde erfolgreich exportiert nach: ./out/attributes/Passer domesticus_attributes.csv
Die Attribute für 20240326_Gartenrotschwanz_Phoenicurus_phoenicurus_AM.xlsx wurde erfolgreich exportiert nach: ./out/attributes/Phoenicurus phoenicurus_attributes.csv
Die Attribute für 20241011_Zwergfledermaus_PipistrellusPipistrellus_AM.xlsx wurde erfolgreich exportiert nach: ./out/attributes/Pipistrellus pipistrellus_attributes.csv
Die Attribute für 20241105_Zweifarbfledermaus_Vespertilio_murinus_CE_AM.xlsx wurde erfolgreich exportiert nach: ./out/attributes/Vespertilio murinus_attributes.csv
Die Attribute für 20241105_Spaltenwollbiene_Anthidium_oblongatum_AM.xlsx wurde erfolgreich exportiert nach: ./out/attributes/Anthidium oblongatum_attributes.csv
Die Attribute für 20230320_